In [1]:
import os
import cv2
import mne
import json
import pickle
import ndx_events
import numpy as np
import matplotlib.pyplot as plt
from pynwb import NWBHDF5IO

In [2]:
with open('../settings.json', "r") as f:
    settings = json.load(f)
    
epoch_folder = settings['epochs_folder']
plot_folder = settings['plots_folder']
nwb_folder = settings['nwb_files_folder']

Let's create some handy converting functions

In [3]:
def sample_to_frame(tll_onset_secs, fps, offset):
    """
    Function that calculates the time-point of the video (in frames) given
     the sample number in the EEG.
    """
    secs_video = tll_onset_secs - offset
    
    return secs_video * fps

In [4]:
def frame_to_sample(frame_number, fps, offset):
    """
    Function that calculates the time-point of the video (in frames) given
     the sample number in the EEG.
    """
    tp_secs_video = frame_number / fps  # time-point on video in seconds
    secs_eeg = tp_secs_video + offset
    
    return secs_eeg

In [5]:
def get_fps(video_path):
    # Open the video file
    video = cv2.VideoCapture(video_path)
    
    # Check if the video is opened successfully
    if not video.isOpened():
        print("Error: Unable to open video file")
        return None
    
    # get the frames per second (FPS)
    fps = video.get(cv2.CAP_PROP_FPS)
    
    # Release the video object
    video.release()
    
    return fps

### Now, let's test and validate the converting process

More sophisticated approach; get all LED onset frame numbers and calculate the seconds/sample num of the EEG. If they are close enough, the method works.

First, let's get all EEG TTL onset timepoints

In [6]:
test_subject = "80630"  # is in batch 1 (see metadata excel sheets)
video_filename = "drd2_batch1_resting-state Camera 1 7-7-2023 09_40_22 1.mp4"

In [10]:
eeg_ttl_onsets_secs = []  # container (to make it accessible through notebook)
s_freq = 0  # placeholder

for file in os.listdir(nwb_folder):
    if test_subject in file and file.endswith(".nwb"):
        with NWBHDF5IO(f'{nwb_folder}/{file}', "r") as io:  # open it
            nwb = io.read()
            eeg_ttl_onsets_secs = list(nwb.acquisition["TTL_1"].timestamps)
            s_freq = nwb.acquisition['raw_EEG'].rate

548948
[nan nan nan ...  0.  0.  0.]
[0.0000000e+00 3.3000000e-02 6.7000000e-02 ... 1.8305010e+04 1.8305044e+04
 1.8305077e+04]


Now that we have the TTL onset timepoints from the EEG of the test_subject, let's load the data that holds the LED ON timepoints from the accompanying video file.

In [92]:
folder_path = "/Users/olledejong/Documents/MSc_Biology/ResearchProject2/rp2_data/resting_state/output/videos"
pickle_path = f"{folder_path}/pickle"

led_states = 0
with open(f'{pickle_path}/led_states_all_videos.pickle', "rb") as f:
    led_states = pickle.load(f)

# movie_name = list(led_states.keys())[0]
# led_states_data = list(led_states.values())[0]

In [93]:
led_states_data = led_states["drd2_batch4_resting-state Camera 1 13-10-2023 09_29_44 1.mp4"]

We only need the frame numbers where the LED switches ON, not all frames where the LED is ON.

In [94]:
led_turns_on_indexes = np.where(np.logical_and(np.diff(led_states_data), led_states_data[1:]))[0] + 1  # get all False to True changes in the array

In [95]:
print(led_turns_on_indexes)

[   220    250    280 548578 548608 548638]


Using the frame number where the LED turns ON for the first time, we can calculate the needed offset.
We calculate this by subtracting the time elapsed between start of **video** and the first LED onset from the time elapsed between start of **EEG** recording and the first TTL onset.

In [96]:
video_fps = 30

first_ttl_onset = eeg_ttl_onsets_secs[0]
first_LED_onset = led_turns_on_indexes[0]

offset = first_ttl_onset - first_LED_onset / video_fps  # TTL onset is in seconds, so we need to transform the LED onset (in frames) to seconds as well (divide by frames per second)
# 19

In [97]:
for i, frame_led_on in enumerate(led_turns_on_indexes):
    eeg_ttl_in_secs = frame_to_sample(frame_led_on, fps=video_fps, offset=offset)
    print(f"Onset {i}. Actual EEG TTL onset: {eeg_ttl_onsets_secs[i]}, calculated: {eeg_ttl_in_secs}. Delta: {eeg_ttl_onsets_secs[i] - eeg_ttl_in_secs}")

Onset 0. Actual EEG TTL onset: 16.0023, calculated: 16.0023. Delta: 0.0
Onset 1. Actual EEG TTL onset: 17.0044, calculated: 17.002300000000005. Delta: 0.0020999999999951058
Onset 2. Actual EEG TTL onset: 18.0065, calculated: 18.002300000000005. Delta: 0.004199999999993764
Onset 3. Actual EEG TTL onset: 18309.3886, calculated: 18294.602300000002. Delta: 14.786299999996118
Onset 4. Actual EEG TTL onset: 18310.3898, calculated: 18295.602300000002. Delta: 14.787499999998545
Onset 5. Actual EEG TTL onset: 18311.3919, calculated: 18296.602300000002. Delta: 14.789599999996426


Now let's try the other way around

In [30]:
for i, eeg_ttl_onset in enumerate(eeg_ttl_onsets_secs):
    frame_of_led_onset = sample_to_frame(eeg_ttl_onset, fps=video_fps, offset=offset)
    print(f"Onset {i}. Actual frame LED onset: {led_turns_on_indexes[i]}, calculated: {frame_of_led_onset}")

Onset 0. Actual frame LED onset: 120, calculated: 120.0
Onset 1. Actual frame LED onset: 150, calculated: 150.033
Onset 2. Actual frame LED onset: 180, calculated: 180.096
Onset 3. Actual frame LED onset: 486, calculated: 486.75600000000003
Onset 4. Actual frame LED onset: 516, calculated: 516.7919999999999
Onset 5. Actual frame LED onset: 546, calculated: 546.828
Onset 6. Actual frame LED onset: 591596, calculated: 592048.026
Onset 7. Actual frame LED onset: 591626, calculated: 592078.089
Onset 8. Actual frame LED onset: 591656, calculated: 592108.152


#### Try with FPS extracted from video

In [31]:
video_fps = get_fps(os.path.join(folder_path, video_filename))
print(f"The FPS extracted from the video is {video_fps}")

The FPS extracted from the video is 29.988787547921188


In [32]:
for i, frame_led_on in enumerate(led_turns_on_indexes):
    eeg_ttl_in_secs = frame_to_sample(frame_led_on, fps=video_fps, offset=offset)
    print(f"Onset {i}. Actual EEG TTL onset: {eeg_ttl_onsets_secs[i]}, calculated: {eeg_ttl_in_secs}")

Onset 0. Actual EEG TTL onset: 13.0716, calculated: 13.073095552570893
Onset 1. Actual EEG TTL onset: 14.0727, calculated: 14.073469440713616
Onset 2. Actual EEG TTL onset: 15.0748, calculated: 15.07384332885634
Onset 3. Actual EEG TTL onset: 25.2968, calculated: 25.277656987912117
Onset 4. Actual EEG TTL onset: 26.298, calculated: 26.27803087605484
Onset 5. Actual EEG TTL onset: 27.2992, calculated: 27.27840476419756
Onset 6. Actual EEG TTL onset: 19744.0058, calculated: 19736.311290989415
Onset 7. Actual EEG TTL onset: 19745.0079, calculated: 19737.311664877558
Onset 8. Actual EEG TTL onset: 19746.01, calculated: 19738.3120387657


In [33]:
for i, eeg_ttl_onset in enumerate(eeg_ttl_onsets_secs):
    frame_of_led_onset = sample_to_frame(eeg_ttl_onset, fps=video_fps, offset=offset)
    print(f"Onset {i}. Actual frame LED onset: {led_turns_on_indexes[i]}, calculated: {frame_of_led_onset}")

Onset 0. Actual frame LED onset: 120, calculated: 119.95515019168475
Onset 1. Actual frame LED onset: 150, calculated: 149.97692540590862
Onset 2. Actual frame LED onset: 180, calculated: 180.02868940768047
Onset 3. Actual frame LED onset: 486, calculated: 486.5740757225309
Onset 4. Actual frame LED onset: 516, calculated: 516.5988498155095
Onset 5. Actual frame LED onset: 546, calculated: 546.6236239084882
Onset 6. Actual frame LED onset: 591596, calculated: 591826.748996004
Onset 7. Actual frame LED onset: 591626, calculated: 591856.8007600058
Onset 8. Actual frame LED onset: 591656, calculated: 591886.8525240076


This does seem to increase the accuracy of converting FPS to sample number.

#### Averaged offset (first and last LED onset)

And now let's try the same but with averaged offset

In [34]:
last_ttl_onset = eeg_ttl_onsets_secs[-1]
last_LED_onset = led_turns_on_indexes[-1]

offset_end = last_ttl_onset - last_LED_onset / video_fps
offset_average = (offset + offset_end) / 2

In [35]:
for i, frame_led_on in enumerate(led_turns_on_indexes):
    eeg_ttl_in_secs = frame_to_sample(frame_led_on, fps=video_fps, offset=offset_average)
    print(f"Onset {i}. Actual EEG TTL onset: {eeg_ttl_onsets_secs[i]}, calculated: {eeg_ttl_in_secs}")

Onset 0. Actual EEG TTL onset: 13.0716, calculated: 16.922076169719755
Onset 1. Actual EEG TTL onset: 14.0727, calculated: 17.922450057862477
Onset 2. Actual EEG TTL onset: 15.0748, calculated: 18.922823946005202
Onset 3. Actual EEG TTL onset: 25.2968, calculated: 29.126637605060978
Onset 4. Actual EEG TTL onset: 26.298, calculated: 30.127011493203703
Onset 5. Actual EEG TTL onset: 27.2992, calculated: 31.12738538134642
Onset 6. Actual EEG TTL onset: 19744.0058, calculated: 19740.160271606564
Onset 7. Actual EEG TTL onset: 19745.0079, calculated: 19741.160645494707
Onset 8. Actual EEG TTL onset: 19746.01, calculated: 19742.16101938285


In [36]:
for i, eeg_ttl_onset in enumerate(eeg_ttl_onsets_secs):
    frame_of_led_onset = sample_to_frame(eeg_ttl_onset, fps=video_fps, offset=offset_average)
    print(f"Onset {i}. Actual frame LED onset: {led_turns_on_indexes[i]}, calculated: {frame_of_led_onset}")

Onset 0. Actual frame LED onset: 120, calculated: 4.528888187940927
Onset 1. Actual frame LED onset: 150, calculated: 34.55066340216481
Onset 2. Actual frame LED onset: 180, calculated: 64.60242740393664
Onset 3. Actual frame LED onset: 486, calculated: 371.14781371878706
Onset 4. Actual frame LED onset: 516, calculated: 401.17258781176565
Onset 5. Actual frame LED onset: 546, calculated: 431.1973619047444
Onset 6. Actual frame LED onset: 591596, calculated: 591711.3227340003
Onset 7. Actual frame LED onset: 591626, calculated: 591741.374498002
Onset 8. Actual frame LED onset: 591656, calculated: 591771.4262620037


#### Calculate average offset (using all LED on events)

Now, let's try to calculate the average offset by incorporating all LED on events

In [37]:
offsets = []
for ttl_onset, led_onset in zip(eeg_ttl_onsets_secs, led_turns_on_indexes):    
    offsets.append(ttl_onset - led_onset / video_fps)

average_offset = sum(offsets) / len(offsets)

print(f"All calculated offsets: {offsets}")
print(f"Average offset: {average_offset}")

All calculated offsets: [9.070104447429108, 9.070830559286383, 9.07255667114366, 9.090743012087884, 9.09156912394516, 9.092395235802439, 16.766109010583023, 16.767835122442193, 16.769561234297726]
Average offset: 11.643522713001953


In [38]:
for i, frame_led_on in enumerate(led_turns_on_indexes):
    eeg_ttl_in_secs = frame_to_sample(frame_led_on, fps=video_fps, offset=average_offset)
    print(f"Onset {i}. Actual EEG TTL onset: {eeg_ttl_onsets_secs[i]}, calculated: {eeg_ttl_in_secs}")

Onset 0. Actual EEG TTL onset: 13.0716, calculated: 15.645018265572846
Onset 1. Actual EEG TTL onset: 14.0727, calculated: 16.64539215371557
Onset 2. Actual EEG TTL onset: 15.0748, calculated: 17.645766041858295
Onset 3. Actual EEG TTL onset: 25.2968, calculated: 27.84957970091407
Onset 4. Actual EEG TTL onset: 26.298, calculated: 28.849953589056792
Onset 5. Actual EEG TTL onset: 27.2992, calculated: 29.850327477199514
Onset 6. Actual EEG TTL onset: 19744.0058, calculated: 19738.88321370242
Onset 7. Actual EEG TTL onset: 19745.0079, calculated: 19739.88358759056
Onset 8. Actual EEG TTL onset: 19746.01, calculated: 19740.883961478703


In [39]:
for i, eeg_ttl_onset in enumerate(eeg_ttl_onsets_secs):
    frame_of_led_onset = sample_to_frame(eeg_ttl_onset, fps=video_fps, offset=average_offset)
    print(f"Onset {i}. Actual frame LED onset: {led_turns_on_indexes[i]}, calculated: {frame_of_led_onset}")

Onset 0. Actual frame LED onset: 120, calculated: 42.82630636179609
Onset 1. Actual frame LED onset: 150, calculated: 72.84808157601998
Onset 2. Actual frame LED onset: 180, calculated: 102.89984557779181
Onset 3. Actual frame LED onset: 486, calculated: 409.44523189264225
Onset 4. Actual frame LED onset: 516, calculated: 439.47000598562084
Onset 5. Actual frame LED onset: 546, calculated: 469.49478007859955
Onset 6. Actual frame LED onset: 591596, calculated: 591749.620152174
Onset 7. Actual frame LED onset: 591626, calculated: 591779.6719161759
Onset 8. Actual frame LED onset: 591656, calculated: 591809.7236801776


### Plotting the events (before and after aligning)

#### Before

In [ ]:
positions = np.array([2, 4, 6])[:,np.newaxis] 
offsets = [2,4,6] 

plt.eventplot(positions, lineoffsets=offsets) 
plt.show() 

#### After

Let's load a single filtered epoch file

In [7]:
for file in os.listdir(epoch_folder):
    if test_subject in file and "filtered" in file and file.endswith(".fif"):
        epochs = mne.read_epochs(os.path.join(epoch_folder, file), preload=True)
        print(epochs)

Reading C:\Users\Olle de Jong\Documents\MSc Biology\rp2\rp2_data\resting_state\output\epochs\filtered_epochs_resting_state_79593-epo.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...    4998.53 ms
        0 CTF compensation matrices available
Adding metadata with 3 columns
1108 matching events found
No baseline correction applied
0 projection items activated
<EpochsFIF |  1108 events (all good), 0 – 4.99853 s, baseline off, ~412.6 MB, data loaded, with metadata,
 '1': 1108>
